## Indian (Tamil) Language Tokenizer

In [ ]:
text = "abc"
text = "அமிழ்து"
tokens = text.encode("utf-8")
tokens = list(map(int, tokens))



In [ ]:
!wget https://archive.org/download/cholloadai-2021.txt/cholloadai-2021.txt.00.gz

--2024-06-18 03:22:42--  https://archive.org/download/cholloadai-2021.txt/cholloadai-2021.txt.00.gz
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia904605.us.archive.org/20/items/cholloadai-2021.txt/cholloadai-2021.txt.00.gz [following]
--2024-06-18 03:22:42--  https://ia904605.us.archive.org/20/items/cholloadai-2021.txt/cholloadai-2021.txt.00.gz
Resolving ia904605.us.archive.org (ia904605.us.archive.org)... 207.241.235.65
Connecting to ia904605.us.archive.org (ia904605.us.archive.org)|207.241.235.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45020436 (43M) [application/octet-stream]
Saving to: ‘cholloadai-2021.txt.00.gz’

cholloadai-2021.txt 100%[===================>]  42.93M  8.04MB/s    in 6.0s    

2024-06-18 03:22:49 (7.10 MB/s) - ‘cholloadai-2021.txt.00.gz’ saved [45020436/45020436]



In [ ]:
!ls cholloadai-2021.txt.00.gz
!gunzip cholloadai-2021.txt.00.gz
!ls

cholloadai-2021.txt.00.gz
cholloadai-2021.txt.00	sample_data


In [ ]:

with open('cholloadai-2021.txt.00', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
tokens = text.encode("utf-8")
tokens = list(map(int, tokens))

In [ ]:


def get_stats(ids):
  counts = {}
  for pair in zip(ids, ids[1:]):
    counts[pair] = counts.get(pair, 0) + 1
  return counts

def merge(ids, pair, idx):
  newids = []
  i = 0
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids

In [ ]:


vocab_size = 300
num_merges = vocab_size - 256
ids = list(tokens)

merges = {}
for i in range(num_merges):
  stats = get_stats(ids)
  pair = max(stats, key=stats.get)
  idx = 256 + i
  print(f"merging {pair} into a new token {idx}")
  ids = merge(ids, pair, idx)
  merges[pair] = idx


print("tokens length:", len(tokens))
print("ids length: ", len(ids))
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")


merging (224, 174) into a new token 256
merging (224, 175) into a new token 257
merging (257, 141) into a new token 258
merging (32, 256) into a new token 259
merging (258, 256) into a new token 260
merging (257, 129) into a new token 261
merging (256, 191) into a new token 262
merging (262, 256) into a new token 263
merging (261, 256) into a new token 264
merging (258, 259) into a new token 265
merging (256, 190) into a new token 266
merging (266, 256) into a new token 267
merging (260, 149) into a new token 268
merging (256, 176) into a new token 269
merging (260, 164) into a new token 270
merging (257, 136) into a new token 271
merging (256, 159) into a new token 272
merging (256, 164) into a new token 273
merging (261, 259) into a new token 274
merging (256, 169) into a new token 275
merging (256, 179) into a new token 276
merging (256, 174) into a new token 277
merging (256, 149) into a new token 278
merging (260, 170) into a new token 279
merging (260, 159) into a new token 280
m

In [ ]:


vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
  vocab[idx] = vocab[p0] + vocab[p1]

def decode(ids):
  tokens = b"".join(vocab[idx] for idx in ids)
  text = tokens.decode("utf-8", errors="replace")
  return text

print(decode([128])) # Expected : it should print Tamil text as output

�


In [ ]:


def encode(text):
  tokens = list(text.encode("utf-8"))
  while len(tokens) >= 2:
    stats = get_stats(tokens)
    pair = min(stats, key=lambda p: merges.get(p, float("inf")))
    if pair not in merges:
      break
    idx = merges[pair]
    tokens = merge(tokens, pair, idx)
  return tokens

print(decode(encode("hello world")))



hello world


In [ ]:
import regex as re
gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

print(re.findall(gpt2pat, "Hello've world123 how's are you!!!?"))